## BRONZE TO SILVER LAYER


In [2]:
import pandas as pd
from common_utilities import replace_punctuation_from_columns
from global_paths import TradeHistoryBronzeLayerPath,TradeHistorySilverFilePath

### Bronze Layer - Trade History


In [3]:
# Initialize an empty list to store DataFrames
df_TradeHistory = []

# Loop through List of all CSV files in the folder
for file_path in TradeHistoryBronzeLayerPath.glob("*.csv"):
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Append the DataFrame to the list
    df_TradeHistory.append(df)

# Concatenate all DataFrames into one
df_TradeHistory = pd.concat(df_TradeHistory, ignore_index=True)
df_TradeHistory.columns = replace_punctuation_from_columns(df_TradeHistory.columns)
df_TradeHistory.dropna(how="all", axis=1, inplace=True)

# Convert 'trade_num' to int
df_TradeHistory["trade_num"] = df_TradeHistory["trade_num"].fillna(0).astype(int)

# Remove the currency symbol and commas, then convert to float
df_TradeHistory["amount"] = (
    df_TradeHistory["amount"].replace("[₹,]", "", regex=True).astype(float)
)

df_TradeHistory["price"] = (
    df_TradeHistory["price"].replace("[₹,]", "", regex=True).astype(float)
)

# Add Datetime Col
df_TradeHistory["datetime"] = pd.to_datetime(
    df_TradeHistory["date"] + " " + df_TradeHistory["trade_time"].fillna("00:00:00"),
    format="%d-%m-%Y %H:%M:%S",
)

# Convert 'expiry' to desired string format
df_TradeHistory["expiry"] = pd.to_datetime(df_TradeHistory["expiry"], format="%d-%m-%Y")
df_TradeHistory["expiry"] = df_TradeHistory["expiry"].dt.strftime("%d%b%Y")

# Convert the 'side' column in df_TradeHistory to uppercase
df_TradeHistory["side"] = df_TradeHistory["side"].str.upper()

# Function to apply the conditional concatenation
def concat_company(row):
    if row["instrument_type"] == "European Call":
        return row["company"] + "-CE-" + str(row["strike_price"]) + "-" + row["expiry"]
    elif row["instrument_type"] == "European Put":
        return row["company"] + "-PE-" + str(row["strike_price"]) + "-" + row["expiry"]
    else:
        return row["company"]

# Apply the function to the DataFrame
df_TradeHistory["company"] = df_TradeHistory.apply(concat_company, axis=1)

# Rename
df_TradeHistory = df_TradeHistory.rename(
    columns={
        "side": "trade_type",
        "quantity": "trade_quantity",
        "price": "trade_price",
    }
)

# sort the dataframe by date
df_TradeHistory = df_TradeHistory.sort_values(by=["datetime", "company"])


df_TradeHistory = df_TradeHistory[
    [
        "datetime",
        "exchange",
        "segment",
        "company",
        "scrip_code",
        "trade_type",
        "amount",
        "trade_quantity",
        "trade_price",
    ]
]

# Save the result as a csv file
df_TradeHistory.to_csv(TradeHistorySilverFilePath, index=None)
df_TradeHistory.info()

print("Silver Layer csv file for trade history successfully created at:")
print(TradeHistorySilverFilePath.resolve())

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 17 to 40
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        166 non-null    datetime64[ns]
 1   exchange        166 non-null    object        
 2   segment         166 non-null    object        
 3   company         166 non-null    object        
 4   scrip_code      166 non-null    object        
 5   trade_type      166 non-null    object        
 6   amount          166 non-null    float64       
 7   trade_quantity  166 non-null    float64       
 8   trade_price     166 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 13.0+ KB
Silver Layer csv file for trade history successfully created at:
C:\Users\prashant.tripathi\Code\Upstox\DATA\SILVER\TradeHistory\TradeHistory_data.csv
